# Part 1: Data import and cleaning

In [1]:
#import initial libraries

import pandas as pd
import numpy as np

In [2]:
#import data 

df = pd.read_csv("data/stimulus.csv")

In [3]:
# look at basic info about data

df.info()
# 26409 Tweets, after retweets removed using Twarc

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26409 entries, 0 to 26408
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            26409 non-null  int64  
 1   tweet_url                     26409 non-null  object 
 2   created_at                    26409 non-null  object 
 3   parsed_created_at             26409 non-null  object 
 4   user_screen_name              26409 non-null  object 
 5   text                          26409 non-null  object 
 6   tweet_type                    26409 non-null  object 
 7   coordinates                   3 non-null      object 
 8   hashtags                      2820 non-null   object 
 9   media                         1894 non-null   object 
 10  urls                          9372 non-null   object 
 11  favorite_count                26409 non-null  int64  
 12  in_reply_to_screen_name       12629 non-null  object 
 13  i

In [4]:
# count number of languages in data set

count_lang = df['lang'].unique()
print(len(count_lang), count_lang)

31 ['en' 'in' 'und' 'et' 'fr' 'es' 'ja' 'tl' 'de' 'ur' 'ro' 'cs' 'zh' 'pt'
 'cy' 'el' 'ht' 'iw' 'nl' 'sv' 'th' 'hi' 'tr' 'ar' 'da' 'no' 'pl' 'ko'
 'fi' 'ru' 'fa']


In [6]:
# tweets are in 31 different languages

# I'll be working only with Tweets in English
# drop tweets in all other languages
# now working with 25386 Tweets 

df = df[df.lang == 'en']
df.shape

(25386, 37)

In [7]:
# drop unnecessary columns 

df = df.drop(['tweet_url', 'created_at', 'media', 'urls','in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'retweet_or_quote_id',
       'retweet_or_quote_screen_name', 'retweet_or_quote_user_id', 'source',
       'user_created_at', 'user_name', 'user_verified', 'user_friends_count', 'user_listed_count',
       'user_statuses_count', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'coordinates', 'lang', 'user_location', 'user_time_zone', 'user_urls', 'place'], axis=1)

In [8]:
# check end time & date of data

df.iloc[0]

# last Tweet downloaded 2021-02-04 00:09:07+00:00

id                                                  1357118953080905728
parsed_created_at                             2021-02-04 00:09:07+00:00
user_screen_name                                          JohnnyRoseATH
text                  @chick7667 @Manpree31938145 Of course there's ...
tweet_type                                                        reply
hashtags                                                            NaN
favorite_count                                                        0
possibly_sensitive                                                False
retweet_count                                                         0
user_id                                             1334316647302262788
Name: 0, dtype: object

In [10]:
# check first time & date of data 

df.iloc[-1]

# first Tweet 2021-02-02 22:05:30+00:00

id                                                  1356725457362636802
parsed_created_at                             2021-02-02 22:05:30+00:00
user_screen_name                                        ColleenJudkins3
text                  @ReverendWarnock \n#MSC \n#SS200 \n#FPUCRETRO ...
tweet_type                                                     original
hashtags                                  MSC SS200 FPUCRETRO HAZARDPAY
favorite_count                                                        2
possibly_sensitive                                                  NaN
retweet_count                                                         2
user_id                                             1297944424484188162
Name: 26408, dtype: object

# Part 2: Text processing for NLP 

In [11]:
# create variable for "text" column 
text = df['text'] 

In [12]:
# tokenize, remove stopwords, remove urls, lowercase, remove punctuation, remove numbers

# import necessary libraries: ntlk etc.

import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 


stop = stopwords.words('english')

punc = list(set(string.punctuation))

def tokenizer(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

def remove_url(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r'', text)

def process_text(text):
    text = remove_url(text)
    text = tokenizer(text)
    text = [word.lower() for word in text]
    text = [re.sub('[0-9]+', '', word) for word in text]
    text = [word for word in text if word not in punc]
    text = [word for word in text if word not in stop]
    text = [each for each in text if len(each) > 1]
    text = [word for word in text if ' ' not in word]
     
    return text

In [13]:
# apply text processing functions to text

df['processed_text'] = df['text'].apply(process_text)

In [14]:
# look at some of processed text

pd.set_option('display.max_colwidth', -1)
df['processed_text'][:20]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


0     [@chick, @manpree, course, there's, foreign, aid, package, last, one, either, normal, federal, budget, foreign, aid, always, passed, time, covid, bill, here's, what's, bill]                                                                            
1     [biden, flexible, gets, aid, tells, lawmakers, go, big, including, stimulus, checks]                                                                                                                                                                     
2     [@rneal, @scheerpa, @gopchairwoman, we'll, see, stimulus, discussions]                                                                                                                                                                                   
3     [@ezrocks, arizona, sues, biden, admin, biden, insists, stimulus, checks, city, declares, donald, trump, week]                                                                                                                    

In [15]:
# part-of-speech tagging 

ready_for_pos = df['processed_text']

def pos_tagging(text):
    pos_tag = [pos_tag(word) for word in ready_for_pos]

df['pos_tagged'] = df.processed_text.apply(lambda x: pos_tag(x))

In [18]:
# lemmatizing

pos_tagged = df['pos_tagged']

from nltk.stem import WordNetLemmatizer 
wordnet = WordNetLemmatizer() 

lemmatized = [[wordnet.lemmatize(word[0]) for word in words] for words in pos_tagged]

In [19]:
# look at lemmatized text

df['lemmatized'] = lemmatized
lemmatized[:20]

[['@chick',
  '@manpree',
  'course',
  "there's",
  'foreign',
  'aid',
  'package',
  'last',
  'one',
  'either',
  'normal',
  'federal',
  'budget',
  'foreign',
  'aid',
  'always',
  'passed',
  'time',
  'covid',
  'bill',
  "here's",
  "what's",
  'bill'],
 ['biden',
  'flexible',
  'get',
  'aid',
  'tell',
  'lawmaker',
  'go',
  'big',
  'including',
  'stimulus',
  'check'],
 ['@rneal',
  '@scheerpa',
  '@gopchairwoman',
  "we'll",
  'see',
  'stimulus',
  'discussion'],
 ['@ezrocks',
  'arizona',
  'sue',
  'biden',
  'admin',
  'biden',
  'insists',
  'stimulus',
  'check',
  'city',
  'declares',
  'donald',
  'trump',
  'week'],
 ['poll',
  'find',
  'rds',
  'american',
  'support',
  'pres',
  "biden's",
  '#covid',
  'relief',
  'plan',
  'support',
  'stimulus',
  'check',
  'provision',
  '@cnbc',
  'rpts',
  'nearly',
  'republican',
  'congress',
  'oppose',
  'also',
  'support',
  'hour',
  'federal',
  'minimum',
  'wage',
  'hike'],
 ['@repdanbishop',
  '@ho

In [20]:
# before vectorizing, cast lists of words back into strings

df['final_docs'] = df['lemmatized'].apply(lambda x: " ".join(x))
pd.set_option('display.max_colwidth', -1)
final_docs = df['final_docs']
final_docs[3000:3020]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


3083    stimulus finally arrived defi put leaning toward grt                                                                                                                                                 
3084    @leadermcconnell @gop year get pandemic control smart enough know previous stimulus money went work know right pas bill chance failed miserably                                                      
3085    faux administration confirmed stimulus zimbabwean goat rancher want measure air speed ... billion dude literally broken every promise week impressive                                                
3086    already receive likely eligible prior stimulus payment well #mycaleitc                                                                                                                               
3087    covid case going drastically need massive covid stimulus bill either                                                                                                    

# Part 3: NMF and LDA modeling

In [23]:
#create document term matrix with TFIDF

#import vectorizing tool (usee TFIDF)
from sklearn.feature_extraction.text import TfidfVectorizer
# set max_features to 2000 (specifies the number of most frequently occurring words for which we want to create feature vectors)
# set min_df to 5 (word must occur in at least 5 documents)
# set max_df to 0.85 (word must not occur in more than 85 percent of the documents) 

tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.85, ngram_range=(1, 2), stop_words='english')  
doc_term_matrix_1 = tfidfconverter.fit_transform(df['final_docs'].values.astype('U'))

In [28]:
#run NMF model 

#import NMF tool 
from sklearn.decomposition import NMF

nmf_model = NMF(n_components=10)
nmf_Z = nmf_model.fit_transform(doc_term_matrix_1)

In [27]:
# run LDA model

#import LDA tool 
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components = 10, max_iter=10, learning_method='online', learning_decay=.5)
lda_Z = lda_model.fit_transform(doc_term_matrix_1)

In [29]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, tfidfconverter )
print("=" * 30)
 
print("NMF Model:")
print_topics(nmf_model, tfidfconverter )
print("=" * 30)

LDA Model:
Topic 0:
[('check', 243.17387319612098), ('tax', 243.0047088244624), ('stimulus check', 216.6961258813581), ('income', 208.67713707347588), ('year', 175.309986196525), ('payment', 139.880465558418), ('need', 139.4357521062503), ('stimulus payment', 139.27672701125337), ('people', 133.40725674442157), ('money', 133.08592022594766)]
Topic 1:
[('stimulus check', 275.501361455345), ('check', 265.6008285066058), ('joe', 164.85449406318537), ('new', 111.84332112011639), ('manchin', 98.37373203805818), ('tell', 97.2237227405877), ('biden', 93.81863940222195), ('youtube', 93.37953447986935), ('joe manchin', 92.4027683433445), ('hope', 83.79468351843153)]
Topic 2:
[('dollar', 158.97466251735355), ('finally', 83.75204911606518), ('gonna', 80.7512906794185), ('check', 79.53937554203253), ('stimulus check', 78.69697585413434), ('increase', 74.25169155458165), ('account', 72.44018277740744), ('million', 68.77937366202619), ('gave', 64.35599932667273), ('understand', 61.443229095289446)]


In [32]:
 # visualization of LDA model 
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, doc_term_matrix_1, tfidfconverter, mds='tsne')
panel


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
7      32.420952  8.115230   1       1        17.816808
0      0.648955   71.707100  2       1        14.697688
5      71.019684  9.971006   3       1        11.407744
3      3.785981   33.185135  4       1        8.792414 
1     -42.666500 -9.222070   5       1        8.405045 
4     -3.621689  -43.526516  6       1        8.042706 
8     -3.632402  -4.171708   7       1        8.026399 
9     -36.584400  35.804413  8       1        7.971685 
2      44.391380  50.615383  9       1        7.893701 
6      40.235809 -31.591665  10      1        6.945809 , topic_info=                  Term        Freq        Total Category  logprob  loglift
1185  package           464.000000  464.000000   Default  30.0000  30.0000
1666  stimulus package  370.000000  370.000000   Default  29.0000  29.0000
511   economic          236.000000  236.000000   Default  28.0000  28.0000
1018  market            194.000000  194.000000   Default  27.0000  27.0000
1899  voted             261.000000  261.000000   Default  26.0000  26.0000
...     ...                    ...         ...       ...      ...      ...
1916  want              77.299242   284.856452   Topic10 -4.2635   1.3627 
864   job               48.761222   216.074677   Topic10 -4.7242   1.1783 
251   check             46.880927   1199.987095  Topic10 -4.7635  -0.5754 
1640  stimulus check    43.636341   1128.227269  Topic10 -4.8353  -0.5855 
1496  say               40.688845   213.532717   Topic10 -4.9052   1.0092 

[482 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3     5      0.979286  absolutely
8     9      0.986137  account   
9     5      0.983157  act       
13    10     0.987115  activity  
16    2      0.981123  actually  
...   ..          ...       ...  
1987  2      0.637485  year      
1987  3      0.079686  year      
1987  7      0.083670  year      
1993  1      0.991126  yes       
1997  5      0.986212  youtube   

[671 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 6, 4, 2, 5, 9, 10, 3, 7])

In [33]:
# make results of LDA1 model a column in dataframe

topic_values = lda_model.transform(doc_term_matrix_1)
df['LDA1_topic'] = topic_values.argmax(axis=1)

In [34]:
df.head(2)

,id,parsed_created_at,user_screen_name,text,tweet_type,hashtags,favorite_count,possibly_sensitive,retweet_count,user_id,processed_text,pos_tagged,lemmatized,final_docs,LDA1_topic
0,1357118953080905728,2021-02-04 00:09:07+00:00,JohnnyRoseATH,@chick7667 @Manpree31938145 Of course there's no foreign aid in this package. There wasn't any in the last one either. That was normal federal budget foreign aid we always have that was passed at the same time as the covid bill. Here's what's in this bill: https://t.co/LawC4ZulvB,reply,NaN,0,False,0,1334316647302262788,"[@chick, @manpree, course, there's, foreign, aid, package, last, one, either, normal, federal, budget, foreign, aid, always, passed, time, covid, bill, here's, what's, bill]","[(@chick, JJ), (@manpree, NNP), (course, NN), (there's, NN), (foreign, JJ), (aid, NN), (package, NN), (last, JJ), (one, CD), (either, CC), (normal, JJ), (federal, JJ), (budget, NN), (foreign, JJ), (aid, NN), (always, RB), (passed, VBD), (time, NN), (covid, NN), (bill, NN), (here's, VBD), (what's, JJ), (bill, NN)]","[@chick, @manpree, course, there's, foreign, aid, package, last, one, either, normal, federal, budget, foreign, aid, always, passed, time, covid, bill, here's, what's, bill]",@chick @manpree course there's foreign aid package last one either normal federal budget foreign aid always passed time covid bill here's what's bill,4
1,1357118943295660034,2021-02-04 00:09:04+00:00,NBCScottsbluff,"Biden flexible on who gets aid, tells lawmakers to ‘go big’, including stimulus checks https://t.co/QyP1RZ1Bkw",original,NaN,0,False,0,2189073337,"[biden, flexible, gets, aid, tells, lawmakers, go, big, including, stimulus, checks]","[(biden, NN), (flexible, JJ), (gets, VBZ), (aid, NN), (tells, NNS), (lawmakers, NNS), (go, VBP), (big, JJ), (including, VBG), (stimulus, NN), (checks, NNS)]","[biden, flexible, get, aid, tell, lawmaker, go, big, including, stimulus, check]",biden flexible get aid tell lawmaker go big including stimulus check,2


In [37]:
df['text'][df.LDA1_topic ==0][:20] #generic

10     @SenSchumer @SpeakerPelosi Please don’t cave to GOP and Please please don’t let @POTUS whittle down the threshold of stimulus checks (poorly named!- it’s a Rescue)! Do you know what rents cost ? Food? Child care if you’re even employed? I voted for you guys. Don’t let us down. ❤️                                   
12     Biden signs executive orders on stimulus checks, minimum wage, and food ... https://t.co/1rrdYtx0Ox via @YouTube                                                                                                                                                                                                           
13     "How to claim missing stimulus payments on your 2020 tax return"  https://t.co/uLL9iQ5GFt                                                                                                                                                                                                                                  
20     @RepMTG @mtgreenee Here’

In [40]:
df['text'][df.LDA1_topic ==1][120:140]  

1516    Report: Democrats Open To Capping Income For Stimulus Checks At $50,000 A Year For Individuals https://t.co/GKzjIqGKVi                                                                                           
1517    Biden open to negotiating terms of $1,400 stimulus checks: Report https://t.co/YTce79pCan                                                                                                                        
1523    Here’s the new Democratic plan for $1,400 stimulus checks - The Washington Post https://t.co/MDHRgC99jt                                                                                                          
1527    Report: Democrats Open To Capping Income For Stimulus Checks At $50,000 A Year For Individuals https://t.co/mOKfhkZWmX                                                                                           
1531    @ibrahim_zbib @umichvoter99 Not if they skimp on the stimulus checks                                                    

In [42]:
df['text'][df.LDA1_topic == 2][120:140]  

1960    Biden shows flexibility but tells House to ‘go big’ on virus aid, including stimulus checks https://t.co/ZVFCiyuUzO                                                                                                                                                                                                      
1971    Just gonna put this here https://t.co/iwT2iP2GBZ https://t.co/8cdJWI6SVg                                                                                                                                                                                                                                                 
1975    @isaac_ru1 @AP False. Used his own stimulus check to give the money to a friend who bought the gun for him. Only reason he didn’t buy himself is because he wasn’t old enough. You’re a shittier defense attorney that fraud Lin wood. Enjoy watching this kid rot in prison. Bye murder advocate.                       
1976    @LeaderMcConnell Time to g

In [43]:
df['text'][df.LDA1_topic ==3][120:140]  # high emotion

1630    Finally got my second stimulus check today https://t.co/fgUQZrbb0Z                                                                                                                                                                                                                                                    
1641    @melwedde Maybe with all the delays they'll be able to base the stimulus off of 2020 returns and not 2019. So many people did well in '19 and were excluded from relief despite getting hit hard in '20                                                                                                               
1645    @MarshaBlackburn The Senator has confused cause and effect.  We wouldn't need a stimulus if there were enough jobs. A good stimulus results in more jobs.                                                                                                                                                             
1661    @SenatorCardin @IsabelCGuzman @SBAg

In [44]:
df['text'][df.LDA1_topic ==3][1000:1020]  # high emotion

14107    Stimulus Checks: COVID-19 Relief Moves Forward Despite No Support From Republicans\n\nhttps://t.co/js1vUMqG16 https://t.co/4vQzh2igAU                                                                                                                                                                            
14118    @semperdiced Manchin wants to means test the one-time Covid stimulus, but not minimum wage (he thinks $11 an hour nationwide is cool). Make it make sense.                                                                                                                                                       
14159    Connecticut for example, has a $13.50 minimum wage, a two dollar increase isn't going to make or break an entire company, and if that's the case, this stimulus bill is aiding in small businesses to ensure that exact thing doesn't happen. if you want to talk inflation, inflation                           
14181    @FoxBusiness @mokylatn tell Brittany De Lea no

In [45]:
df['text'][df.LDA1_topic ==3][1050:1070]  # high emotion

14889    @Jim_Jordan So the best stimulus is a job but you’d rather have every teacher in Chicago lose their jobs because they *gasp* want to stay safe? Oh the horror...                                                                                                                                                         
14890    Sorry but if senate democrats wanted a stimulus to pass they wouldn’t have put a 15 dollar minimum wage in the bill. Democrats don’t care about you anymore than republicans. The sooner you people realize this the better.                                                                                             
14894    @Bri71947477 @jokerbit3h @turbotax I'm not saying it was her fault just saying she shouldn't have used TT. If you keep using them and something like this happens again then it would be your fault. I didn't know my stimulus and tax refund would get messed up but I'm going to giving them the chance to do it again.
14906    Where’s my fucking sti

In [46]:
df['text'][df.LDA1_topic ==4][1050:1070]  

20578    ... wait ... FIVE COVID STIMULUS PACKAGES ?!\n\nwe’ve only seen 2 &amp; i’ve only seen information about 3 (the initial two included) ... so where the OTHER TWO PACKAGES AT ?!-\n\nthe American people have received $1800&lt; SINCE THE DAMN PANDEMIC BEGAN OUT OF FIVE PASSED PACKAGES ?!- https://t.co/uL32aaReoS
20597    @SenatorHagerty Don't even utter the word "partisan" after the past 4 years of Repugnant partisan 🐂💩  . It was late summer thru late fall Repugnants drug their feet on stimulus.  You only now plead for "bipartisanship" since you lost.                                                                           
20599    @TheDemocrats Asking . Will there be retroactive unemployment in this next stimulus aid package? It was promised to all Americans  and was brought up everyday on the campaign trail.....not as much lately! Let's pass retroactive unemployment! Thank you!                                                         
20613    The Era of Neoliberal Svengalis Li

In [47]:
df['text'][df.LDA1_topic ==4][100:120]  #rich /poor

1301    A series of announcements have been made regarding CFAP funding recently. Below, K·Coe’s Farm Program Services team has summarized what you need to know about the temporary pause on CFAP assistance.\n\nhttps://t.co/pwcpjTfUt1 https://t.co/mxohfKAg7t                                                      
1337    @POTUS @VP @ewarren @BernieSanders @SenSchumer #BidenHarris #PURPLEPOWER #SS200 #MSC #FPUCRETRO #LALATE I am mentally/physically disabled &amp; lost a son to suicide. I wake every day lost. My only thing now is vaccine, ssdi 200, stimulus, gov't keep promises. Don't let us down!! 🇺🇸                    
1363    Dillon approves gift card stimulus program to support local businesses https://t.co/1Uun04fXRO                                                                                                                                                                                                                 
1366    @Matthew03066378 @SenateDems @POTUS @SenateGOP He was el

In [48]:
df['text'][df.LDA1_topic ==4][230:250]  #rich /poor

3698    @RepDrewFerguson @HouseGOP @HouseDemocrats Republicans refuse to help struggling Americans. But are ready to help the top 1%\n\nRepublicans against;\n➡️ A strong stimulus package\n➡️ Alternate energy sources. (GOP push fossil fuel for the kickbacks from oil companies)\n➡️ Protecting the environment\nhttps://t.co/awaHMxBDCN
3711    @POTUS @JoeBiden @KamalaHarris @SenSchumer @SpeakerPelosi \n50k is the cut off? That’s less than 24.00 an hour. You campaigned on championing the little guy and the first chance you get, you fuck us? So glad I manned those phone banks. \n\nhttps://t.co/eomUx3ybJM                                                             
3733    Repurpose Unspent Stimulus $$$,Keep Your Campaign Stimulus Promises $1,400+$2,000 Or $2,000 Month Til Covid Over https://t.co/Osb6c6rgjg. @JoeBiden @KamalaHarris @SenateDems @AssemblyDems @HouseDemocrats @ACLU @SenSchumer @ReverendWarnock                                                                                      
3

In [49]:
df['text'][df.LDA1_topic ==5][230:250] 

2159    @DithYoder @kenvogel @GOPLeader Agreed. I didn’t properly convey that I’m all for her side-eye and shade and snark. I just don’t feel that same energy and outrage about the Dems keeping their promise to deliver $2000 stimulus checks. That’s going to go a whole lot further than popping a Q by McCarthy’s name.
2162    I hope they learned from the 2009 stimulus that they'll catch way more flack for a bill that's too small than they would if Republicans say it's too big (they'll always say it's too big). https://t.co/nHejCWkr63                                                                                                  
2166    Gazette-Mail editorial: Justice, Biden, Harris and COVID-19 #bailout #Coronavirus Stimulus Package #Relief #Covid-19 https://t.co/ptgFL6Hhfl                                                                                                                                                                         
2205    Here's how American stimulus checks st

In [50]:
df['text'][df.LDA1_topic ==5][1000:1020] 

8681    White House economist rejects GOP senators' push for $1,000 stimulus checks instead of $1,400 https://t.co/o9mHLmdlce via @Yahoo                                                                                                                                                                                                           
8686    @sammy_shear @Armstrongathome @socialiststeve6 @JoeBiden @PeoplesParty_US And Biden does not have a reason to change the amount or delay the checks at all. He and Dems could pass the stimulus checks without Republican input by using the reconciliation  process.                                                                      
8695    The American people are still waiting for their stimulus relief. Enough compromising.                                                                                                                                                                                                                                   

In [52]:
df['text'][df.LDA1_topic ==5][500:520] #topic 5 is comments to politicians

4456    #Biden’s #COVIDー19 #ReliefPlan has a #MedicaidExpansion #problem  https://t.co/ljJYlY6YtF via @voxdotcom #News #HealthCare #COVID #COVID_19 #COVID19 #stimulusBill #StimulusCheck                                                                                                                                                                                    
4457    One Trillion In Stimulus Money Still Unspent – Lunch Alert! https://t.co/fO3IjV35AG                                                                                                                                                                                                                                                                                  
4466    for the amount of times “$2,000 stimulus checks” has been said in this vid, multiply that by 2,000 and that’s the amount the american people should be receiving from these democrats https://t.co/8Yx01WeNFm                                                       

In [54]:
df['text'][df.LDA1_topic ==6][1000:1020]  

22946    Recent market activity reflects new dynamics. Chief Market Strategist David Joy sums up prices, Q4 earnings and a potential stimulus package.   https://t.co/LIqF1mEZWb
22948    Recent market activity reflects new dynamics. Chief Market Strategist David Joy sums up prices, Q4 earnings and a potential stimulus package.   https://t.co/00fTvYdAjE
22957    Recent market activity reflects new dynamics. Chief Market Strategist David Joy sums up prices, Q4 earnings and a potential stimulus package.   https://t.co/C9VxmpAj71
22963    I don't want the stimulus proposals explained to me. I just want to know what and when the deposit will be.                                                            
22986    @CBSNews Waste of time...the whole congress are useless, they need to give Americans $2000 stimulus check                                                              
22990    Recent market activity reflects new dynamics. Chief Market Strategist David Joy sums up prices, Q4 earning

In [56]:
df['text'][df.LDA1_topic ==7][1500:1520]  

6050    @ericclee1 Not realistic, really. I’m supposed to buy things I don’t want/need? Giving money to people who don’t need it not a real stimulus. Most people will save it instead, widening the wealth gap.                                                                                                                         
6057    @SirPrinceJHB Now u know I don’t agree with that my brother. The call we meant to have is now lunch! 😅 \nBut these policies are just Dom. That stimulus package is a big waste, and the smaller ones they’re sneaking in are going to hurt many and benefit few.                                                                 
6060    Imagine the US government/senate playing funny buggers with peoples lives here. Pissing around with farty stimulus packages. Meant to serve their citizens.\n\nQuick enough to bail out Wall St.\n\nMakes you think the tit's in charge of us do something remotely right albeit not much.                                       
6062    #s

In [59]:
df['text'][df.LDA1_topic ==7][500:520]  # debating specifics of stimulus check amounts

2215    @RashidaTlaib @indig4ever Give ppl the option to decline/opt-out of receiving stimulus if they don't want/need it (give em tax credit) &amp; roll those funds back into a 'Standalone' $2k/mo bill--and start there.  We've been twisting in the wind through 10 months of utter neglect. PLZ Do the right thing by us                                                                                
2218    Why are Dems negotiating with Republicans on the stimulus checks who thought a $1200 check was enough to live on for ten weeks\n\nThey have shown us the ENTIRE four fucking years that they don't give a shit about Americans.                                                                                                                                                                       
2223    @AugustinKeaton @VP If she stands for all stands for all Americans, by the way I have voting for her 3 times now, than she should not let the stimulus bill remove people and families that are po

In [63]:
df['text'][df.LDA1_topic ==8][120:140]  #mcconnell #fave tweet in here

2420    @DoYouEvenLif Or a Stimulus Check Force                                                                                                                                                                                                                                                                                        
2423    McConnell slams Democrats for using the same mechanism to force through COVID-19 stimulus that he used to pass the 2017 tax cuts https://t.co/IWTMPwg3Xy                                                                                                                                                                       
2429    @JohnCornyn @AAF The lack of food in the average Americans house after record unemployment and lack of stimulus during a Pandemic suggest otherwise.                                                                                                                                                                           
2431    Stock ma

In [66]:
df['text'][df.LDA1_topic ==9][:20]

15     @Charles93149380 No stimulus just my personal money. I’m already holding 780k #Doge just wanna show more support on 4th of feb. #Dogecoin🚀🚀                                                                                                                                                             
22     Gold Price Analysis: XAU/USD clings to mild gains above $1,830 amid US stimulus chatters https://t.co/SxecUzLma9 #Gold #Commodities #FiscalPolicy #Coronavirus #Macroeconomics                                                                                                                          
32     House votes to speed stimulus passage after Biden meets with Dems https://t.co/ka3FEPG3iv via @politico                                                                                                                                                                                                 
44     #$1400 is NoT th #$2000StimulusChrcks promised to Georgians of thry voted you in 